<a href="https://colab.research.google.com/github/mperlowin/SoluBuild/blob/main/Voice_Sales_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!python --version

Python 3.10.13


In [5]:
!python -m venv venv
!source venv/bin/activate

In [ ]:
%pip install -r requirements.txt

In [1]:
from google.colab import userdata
from flask import Flask, request, jsonify, Response
from twilio.twiml.voice_response import VoiceResponse
import ngrok
import asyncio
import nest_asyncio
import os
import signal

class FlaskApp:
    def __init__(self):
        # Initialize Flask app
        self.app = Flask(__name__)
        self.port = 5000
        self.setup_routes()
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(self.setup_ngrok())
        self.start_flask_app()

    def setup_routes(self):
        @self.app.route('/', methods=['GET'])
        def home():
            return "Hello, World!"
        
        @self.app.route('/twiml', methods=['GET', 'POST'])
        def twiml():
            response = VoiceResponse()
            response.stream(url=self.listener.url().replace("https", "wss") + "/audio")
            return Response(str(response), mimetype='text/xml')

        @self.app.route('/audio', methods=['POST'])
        def process_audio():
            data = request.data
            with open('audio_stream.raw', 'wb') as f:
                f.write(data)
            return jsonify({"message": "Data received successfully"}), 200
        
        @self.app.route('/shutdown', methods=['POST', 'GET'])
        async def shutdown():
            await self.listener.close()
            print("Tunnel closed.")
            shutdown_server()
            return 'Server shutting down...'
    
    async def setup_ngrok(self):
        token = userdata.get("") if IN_COLAB else os.getenv("")
        ngrok.set_auth_token(token)
        self.listener = await ngrok.forward(f"localhost:{self.port}")
        print(f"Ingress established at {self.listener.url()}")
            

    def start_flask_app(self):
        self.app.run(port=self.port)
        print(f"Flask app running on port: {self.port}.")

def shutdown_server():
    pid = os.getpid()
    os.kill(pid, signal.SIGINT)

import threading

# Global variable to store the FlaskApp instance lol
flask_app_instance = None

def run_flask_app():
    global flask_app_instance
    flask_app_instance = FlaskApp()

# Create a thread targeting the run_flask_app function
flask_thread = threading.Thread(target=run_flask_app)

# Start the thread
flask_thread.start()

Ingress established at https://e9dc-20-171-127-68.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [1]:
from google.colab import userdata
from twilio.rest import Client

account_sid = os.environ[""]
auth_token = os.environ[""]
client = Client(account_sid, auth_token)

call = client.calls.create(
    url="",
    to="+12393513986",
    from_="+18449454174",
)

print(call.sid)

SSLError: HTTPSConnectionPool(host='api.twilio.com', port=443): Max retries exceeded with url: /2010-04-01/Accounts/AC497849ba855523cf7ff0c96352c63cc2/Calls.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))

In [ ]:
# To stop the app
!curl -X POST http://127.0.0.1:5000/shutdown

In [ ]:
def check_threads():
  threads = threading.enumerate()
  print(f"Active Threads ({len(threads)})")
  for thread in threads:
    print(f"- Name: {thread.name}, Alive: {thread.is_alive()}")

In [ ]:
check_threads()

Active Threads (5)
- Name: MainThread, Alive: True
- Name: Thread-2 (_thread_main), Alive: True
- Name: Thread-3, Alive: True
- Name: Thread-1, Alive: True
- Name: _colab_inspector_thread, Alive: True


In [ ]:
stop_flask()

Flask server is not running.


In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/dialogstudio-t5-3b-v1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/dialogstudio-t5-3b-v1.0", device_map="auto")

input_text = "Hi, how are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

I'm doing well, thank you. How are you?


In [ ]:
whisper_tokenizer = AutoTokenizer.from_pretrained("openai/whisper-large-v3")
whisper_model = AutoModelForSeq2SeqLM.from_pretrained("openai/whisper-large-v3")

